In [1]:
import matplotlib.pyplot as plt
import datetime
import numpy as np
import types
import pickle
import os
import glob
import rasterio
import obspy
from obspy.signal.cross_correlation import correlate, xcorr_max
from pyproj import Proj,transform,Geod
from scipy.signal import find_peaks,hilbert,butter,filtfilt
from scipy.fft import rfft,rfftfreq
from scipy.integrate import odeint
from scipy.ndimage.morphology import binary_fill_holes
from scipy.ndimage import label
from skimage.transform import resize
from skimage.morphology import dilation, erosion
from image_analysis.fracture_mapping import clip, align_images, binary_threshold
from seismic_analysis.preprocessing import download_data,remove_ir
from seismic_analysis.visualization import make_spectrogram
from seismic_analysis.attenuation import estimate_Q
from seismic_analysis.location import (compute_backazimuths,get_station_lon_lat,get_station_crs_locations,
                                get_station_grid_locations,load_data,get_grid,get_arrivals,get_velocities,
                                gridsearch,transform_imagery,plot_imagery_seismic_location)
from seismic_analysis.directivity import get_velocity, get_characteristic_frequency
from modeling.viscoelastic_beam import get_coefficients,asymptotic_solution,decay_comparison_plot
from modeling.fluid_fracture import find_min_rxx,model_fracture

In [2]:
'''

Locate the event using travel time inversion of arrivals at regional stations

''' 

# read data
resps = ["DISP","VEL"]
ref_stations = ["PIG2","PIG4","PIG5"]

for resp in resps:
    for ref_station in ref_stations:

        st = obspy.read("data/MSEED/no_IR/*/*/2012-05-09*"+resp+"*")

        # set frequency band for analysis
        freq = [1.5,5]

        # choose component to use for location
        components = ["Z","N","E"]

        # set starttime and endtime for analysis
        starttime=obspy.UTCDateTime(2012,5,9,18,1)
        endtime=obspy.UTCDateTime(2012,5,9,18,9)

        # set on-ice phase velocity
        local_velocity = 1500

        # define grid origin in lat,lon and grid dimensions in meters
        origin_lat_lon = [-75.5,-103.2]
        grid_length = 1.5e5
        grid_height = 1.3e5
        step = 1000
        t0 = -10
        t_step = 0.1

        # select, filter, and trim regional data
        local_stations = ["PIG2","PIG4","PIG5"]
        regional_stations = ["THUR","DNTW","BEAR","UPTW"]
        st_local, st_regional = load_data(st,local_stations,regional_stations,components,freq,starttime,endtime)

        # estimate phase velocity of waves from local reference station to regional stations
        velocities = get_velocities(ref_station,st_local,local_stations,st_regional,regional_stations,components,local_velocity)

        # combine into one stream and remove any stations, if desired
        stations = local_stations + regional_stations
        st = st_local + st_regional

        # get arrival times relative to PIG2
        arrivals, xcorr_coefs = get_arrivals(ref_station,st,stations,components)
        #weights = np.abs(xcorr_coefs)

        # set grid points and compute distance from grid origin to each station
        x_vect,y_vect,t_vect = get_grid(grid_length,grid_height,step,t0,t_step)
        station_lat_lon = np.fliplr(get_station_lon_lat("data/XML/",["XC","YT"],stations))
        station_x_y = get_station_grid_locations(origin_lat_lon,station_lat_lon)

        # carry out the gridsearch
        rmse_mat = gridsearch(t_vect,x_vect,y_vect,station_x_y,velocities,arrivals)

        # find lowest error lat, lon, and origin time
        loc_idx = np.unravel_index(np.argmin(rmse_mat), rmse_mat.shape)
        rmse_loc = rmse_mat[loc_idx[0],:,:]

        # save error matrix
        rmse_file = open("outputs/locations/"+"_".join(stations)+"_ref:"+ref_station+"_"+resp+"_gridsearch.pickle", "wb")
        pickle.dump(rmse_mat, rmse_file)
        rmse_file.close()